# Zero to Hero with TensorFlow (TF Fall 2020 Updates)

## Optional - Check if TensorFlow is available

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

In [8]:
import os
import zipfile

local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')
zip_ref.close()

## Import TensorFlow

In [9]:
import tensorflow as tf

In [11]:
model = tf.keras.models.Sequential([
    # Note the input_Shape is the desired size of the Image 150x150 with 3 bytes color

    # This is the First Convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # This is the Second Convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # This is the Third Convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # This is the Fourth Convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    
    # 512 Neuron Hidden Layer
    tf.keras.layers.Dense(512, activation='relu'),
    
    # Only 1 output neuron. 
    # It will contain a value between 0...1 where 0 for class 1 (Horses) and 1 for class 2 (Human)
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

In [16]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(lr=0.001),
    metrics=['acc']
)

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All Images will be Augmented
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
    './horse-or-human/',         # Source Directory for Training Images
    target_size=(150, 150),      # All images will be resized to 150x150
    batch_size=128,
    # Since we used binary_crossentropy, we need binary labels
    class_mode = 'binary',
)

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
    './horse-or-human/',         # Source Directory for Validation Images
    target_size=(150, 150),      # All images will be resized to 150x150
    # Since we used binary_crossentropy, we need binary labels
    class_mode = 'binary',
)

Found 1027 images belonging to 2 classes.
Found 1027 images belonging to 2 classes.


In [23]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data=validation_generator
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
8/8 [==============================] - 17s 2s/step - loss: 1.0509 - acc: 0.5206 - val_loss: 0.7542 - val_acc: 0.4869
Epoch 2/15
8/8 [==============================] - 16s 2s/step - loss: 0.7043 - acc: 0.5384 - val_loss: 0.6779 - val_acc: 0.5492
Epoch 3/15
8/8 [==============================] - 17s 2s/step - loss: 0.6780 - acc: 0.5873 - val_loss: 0.6565 - val_acc: 0.6660
Epoch 4/15
8/8 [==============================] - 19s 2s/step - loss: 0.6707 - acc: 0.6045 - val_loss: 0.6487 - val_acc: 0.6534
Epoch 5/15
8/8 [==============================] - 16s 2s/step - loss: 0.6887 - acc: 0.6307 - val_loss: 0.6191 - val_acc: 0.7390
Epoch 6/15
8/8 [==============================] - 19s 2s/step - loss: 0.5809 - acc: 0.7100 - val_loss: 0.4941 - val_acc: 0.7303
Epoch 7/15
8/8 [==============================] - 17s 2s/step - loss: 0.6289 - acc: 0.6585 - val_loss: 0.4967 - val_acc: 0.8053
Epoch 8/15
8/8 [============

In [35]:
import numpy as np
from keras.preprocessing import image

while True:
    file = input('Image: > ')
    if file == 'quit':
        break

    img = image.load_img(file, target_size=(150,150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    image_tensor = np.vstack([x])

    classes = model.predict(image_tensor)
#     print(classes)
#     print(classes[0])
    if classes[0] > 0.5:
        print('Human')
    else: print('Horse')

Image: >  test.jpg


Human


Image: >  test1.jpg


Horse


Image: >  quit
